# kNN Hash Example

In [31]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter
from scipy.spatial import distance
from math import sqrt, ceil

## Iris dataset

In [32]:
df = load_iris()
df.data.shape

(150, 4)

In [33]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

* https://docs.python.org/2/library/functools.html Here you can read about "partial"
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html About mapping to [0,1]

In [34]:
def minkowski(v, u, p):
    res = 0
    
    for i in range(v.shape[0]):
        res += pow(v[i] - u[i], p)
    
    return pow(res, 1/p)

def euclid(v, u):
    return minkowskiDist(v, u, 2)

def manhattan(v, u):
    res = 0
    
    for i in range(v.shape[0]):
        res += abs(v[i] - u[i])
        
    return res

In [35]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1, X[0].shape[0]) # weights of a hash function
                f_hh.append(partial(f_hash, w = w, r = random(), b = random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) 
                    # calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n], y[n])) 
                # saving sample into corresponding index
    
    def predict(self,u):
        results = []
        for j in range(self.L):
            inds = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            
            dists = []
            
            for v, label in self.t_hh[j][0][sum(inds)]:
                dists.append([manhattan(v, u), label])
                
            dists.sort()
            
            results.append(dists[0][1])
        return Counter(results).most_common()[0][0]
            
            #Here you must put your code, extend the method with distance function and 
            # calculation with unknown sample "u"
            #Develop the rest part of kNN predict method that was discussed at the lecture

In [36]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target


In [37]:
knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print ('Prediction: ', test1x, ' is ', knnhash.predict(test1x), ' actual value = ', test1y)
print("-------------")

print ('Prediction: ', test2x, ' is ', knnhash.predict(test2x), ' actual value = ', test2y)
print("-------------")

print ('Prediction: ', test3x, ' is ', knnhash.predict(test3x), ' actual value = ', test3y)



Prediction:  [ 0.22222222  0.625       0.06779661  0.04166667]  is  0  actual value =  0
-------------
Prediction:  [ 0.63888889  0.41666667  0.57627119  0.54166667]  is  1  actual value =  1
-------------
Prediction:  [ 0.44444444  0.41666667  0.69491525  0.70833333]  is  2  actual value =  2


* Each string above corresponds to the particular hash table. And index in counter maps to the class. For example Counter({0: 13, 1: 1}) means that there are 13 samples close to "u" with "0" class labels and 1 sample with "1" class label.